# LELEC2870 Machine Learning Project: Predicting a film's gross revenue

## Pauline Terlinden

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.base import BaseEstimator
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
import scipy

import torch
import torch.nn as nn
import torch.nn.functional as F
%matplotlib inline

In [2]:
X1 = pd.read_csv("X1.csv", index_col=0)
X1.reset_index(drop=True, inplace=True)
Y1 = pd.read_csv("Y1.csv", header=None, names=['revenue'])

X1.info()
#Y1.info()

X1.head(10)
#Y1.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3540 entries, 0 to 3539
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            3540 non-null   object 
 1   img_url          3540 non-null   object 
 2   description      3540 non-null   object 
 3   ratings          3540 non-null   float64
 4   n_votes          3540 non-null   float64
 5   is_adult         3540 non-null   int64  
 6   production_year  3540 non-null   int64  
 7   runtime          3540 non-null   object 
 8   genres           3540 non-null   object 
 9   release_year     3540 non-null   float64
 10  studio           3540 non-null   object 
 11  img_embeddings   3540 non-null   object 
 12  text_embeddings  3540 non-null   object 
dtypes: float64(3), int64(2), object(8)
memory usage: 359.7+ KB


,title,img_url,description,ratings,n_votes,is_adult,production_year,runtime,genres,release_year,studio,img_embeddings,text_embeddings
0,Letters to Juliet,https://m.media-amazon.com/images/M/MV5BMjg0OT...,Letters to Juliet: Directed by Gary Winick. Wi...,6.5,92937.0,0,2010,105,"Adventure,Comedy,Drama",2010.0,Sum.,"[0.25030804, 2.4058464, 1.0431569, 0.030648155...","[-0.6795498, 0.35658365, 0.9994932, -0.9793934..."
1,Veil of Tears,https://m.media-amazon.com/images/M/MV5BZjMxOD...,Veil of Tears: Directed by William Gereghty. W...,7.9,11.0,0,1996,\N,"Action,Crime,Drama",2014.0,WF,"[0.51250213, 2.8152602, 0.46308166, 0.29031387...","[-0.6202415, 0.31657028, 0.9992422, -0.9703722..."
2,International Velvet,https://m.media-amazon.com/images/M/MV5BOGVkYj...,International Velvet: Directed by Bryan Forbes...,5.9,1345.0,0,1978,127,"Drama,Family,Sport",1978.0,MGM,"[0.18073043, 0.24735461, 0.63652813, 0.2496522...","[-0.709996, 0.4233521, 0.99980927, -0.98892415..."
3,8 Seconds,https://m.media-amazon.com/images/M/MV5BYjY4Nz...,8 Seconds: Directed by John G. Avildsen. With ...,6.6,4851.0,0,1994,105,"Biography,Drama,Sport",1994.0,NL,"[0.025015268, 0.9105338, 0.3878257, 0.3421247,...","[-0.7416838, 0.38435012, 0.9998453, -0.9874693..."
4,Penitentiary II,https://m.media-amazon.com/images/M/MV5BNjQyZW...,Penitentiary II: Directed by Jamaa Fanaka. Wit...,4.1,549.0,0,1982,108,"Crime,Drama,Sport",1982.0,MGM,"[0.19079691, 1.9068279, 0.29114372, 0.19527505...","[-0.65501904, 0.3845747, 0.9996712, -0.9766391..."
5,The Boondock Saints,https://m.media-amazon.com/images/M/MV5BNTliMT...,The Boondock Saints: Directed by Troy Duffy. W...,7.8,228296.0,0,1999,108,"Action,Crime,Thriller",2000.0,Indic.,"[0.41613612, 1.580189, 0.99092495, 0.55354017,...","[-0.77542937, 0.4372149, 0.9999121, -0.9899120..."
6,Frequency,https://m.media-amazon.com/images/M/MV5BMmFmYj...,Frequency: Directed by Dennis Donnelly. With R...,8.7,68.0,0,1973,60,"Action,Adventure,Comedy",2000.0,NL,"[0.3009643, 1.5369564, 1.6404945, 0.98291993, ...","[-0.6434461, 0.3129971, 0.9994885, -0.9728126,..."
7,Chicago Joe and the Showgirl,https://m.media-amazon.com/images/M/MV5BNDViMT...,Chicago Joe and the Showgirl: Directed by Bern...,5.0,645.0,0,1990,103,"Crime,Drama,Romance",1990.0,NL,"[0.15761384, 1.9583508, 0.6007317, 0.44897583,...","[-0.6398581, 0.35474968, 0.9995949, -0.9757302..."
8,Deuces Wild,https://m.media-amazon.com/images/M/MV5BOTM2Mz...,Deuces Wild: Directed by Scott Kalvert. With S...,5.7,7031.0,0,2002,96,"Action,Crime,Drama",2002.0,MGM,"[0.20481408, 2.2253098, 0.21197028, 0.03752036...","[-0.62410355, 0.3616503, 0.9995014, -0.9757393..."
9,Change of Plans,https://m.media-amazon.com/images/M/MV5BMDFhZj...,Change of Plans: Directed by Randy Zisk. With ...,8.2,12.0,0,1992,\N,"Crime,Drama",2010.0,IFC,"[0.20134872, 3.3128614, 1.3217659, 0.05505188,...","[-0.7538657, 0.43110898, 0.9997868, -0.9895797..."


# 1. Data preprocessing

## 1.1 Deletion of irrelevant variables and missing values

In [3]:
# Dropping title (irrelevant), img_url and description (unnecessary since we have the associated embeddings), is_adult (only 0's)
df = X1.drop(columns=['img_url', 'description', 'title', 'is_adult'])

#print(len(X1) - X1.count())

df.dropna(inplace=True)
# Watch out : for 'runtime', no explicit NaN's, but some values are '\N'
df = df[df.runtime != '\\N']

print(df.head(10))

    ratings   n_votes  production_year runtime                   genres  \
0       6.5   92937.0             2010     105   Adventure,Comedy,Drama   
2       5.9    1345.0             1978     127       Drama,Family,Sport   
3       6.6    4851.0             1994     105    Biography,Drama,Sport   
4       4.1     549.0             1982     108        Crime,Drama,Sport   
5       7.8  228296.0             1999     108    Action,Crime,Thriller   
6       8.7      68.0             1973      60  Action,Adventure,Comedy   
7       5.0     645.0             1990     103      Crime,Drama,Romance   
8       5.7    7031.0             2002      96       Action,Crime,Drama   
10      4.4      84.0             2001      89                 Thriller   
11      7.2  132769.0             1997     113     Comedy,Drama,Romance   

    release_year  studio                                     img_embeddings  \
0         2010.0    Sum.  [0.25030804, 2.4058464, 1.0431569, 0.030648155...   
2         1978.0

## 1.2 Basic manipulation of some features to make them more usable

In [4]:
print(df.dtypes)

ratings            float64
n_votes            float64
production_year      int64
runtime             object
genres              object
release_year       float64
studio              object
img_embeddings      object
text_embeddings     object
dtype: object


### 1.2.1 Replacing the genres column with one binary value for each possible genre

In [5]:
genres = set()
for string in df['genres']:
    genres_loc = string.split(",")
    for genre in genres_loc:
        if genre not in genres:
            genres.add(genre)
            
genres.remove('\\N')
print(genres)
genres = list(genres)
for genre in genres:
    df[genre] = np.where(df.genres.str.contains(genre), 1, 0)


df.drop(columns=['genres'], inplace=True)
df.head(10)

{'Sci-Fi', 'Adventure', 'Game-Show', 'Short', 'Horror', 'War', 'Animation', 'Romance', 'Crime', 'Comedy', 'Thriller', 'Mystery', 'Reality-TV', 'History', 'Biography', 'Western', 'Family', 'Musical', 'Action', 'Film-Noir', 'Documentary', 'Music', 'Fantasy', 'Drama', 'Sport'}


,ratings,n_votes,production_year,runtime,release_year,studio,img_embeddings,text_embeddings,Sci-Fi,Adventure,...,Western,Family,Musical,Action,Film-Noir,Documentary,Music,Fantasy,Drama,Sport
0,6.5,92937.0,2010,105,2010.0,Sum.,"[0.25030804, 2.4058464, 1.0431569, 0.030648155...","[-0.6795498, 0.35658365, 0.9994932, -0.9793934...",0,1,...,0,0,0,0,0,0,0,0,1,0
2,5.9,1345.0,1978,127,1978.0,MGM,"[0.18073043, 0.24735461, 0.63652813, 0.2496522...","[-0.709996, 0.4233521, 0.99980927, -0.98892415...",0,0,...,0,1,0,0,0,0,0,0,1,1
3,6.6,4851.0,1994,105,1994.0,NL,"[0.025015268, 0.9105338, 0.3878257, 0.3421247,...","[-0.7416838, 0.38435012, 0.9998453, -0.9874693...",0,0,...,0,0,0,0,0,0,0,0,1,1
4,4.1,549.0,1982,108,1982.0,MGM,"[0.19079691, 1.9068279, 0.29114372, 0.19527505...","[-0.65501904, 0.3845747, 0.9996712, -0.9766391...",0,0,...,0,0,0,0,0,0,0,0,1,1
5,7.8,228296.0,1999,108,2000.0,Indic.,"[0.41613612, 1.580189, 0.99092495, 0.55354017,...","[-0.77542937, 0.4372149, 0.9999121, -0.9899120...",0,0,...,0,0,0,1,0,0,0,0,0,0
6,8.7,68.0,1973,60,2000.0,NL,"[0.3009643, 1.5369564, 1.6404945, 0.98291993, ...","[-0.6434461, 0.3129971, 0.9994885, -0.9728126,...",0,1,...,0,0,0,1,0,0,0,0,0,0
7,5.0,645.0,1990,103,1990.0,NL,"[0.15761384, 1.9583508, 0.6007317, 0.44897583,...","[-0.6398581, 0.35474968, 0.9995949, -0.9757302...",0,0,...,0,0,0,0,0,0,0,0,1,0
8,5.7,7031.0,2002,96,2002.0,MGM,"[0.20481408, 2.2253098, 0.21197028, 0.03752036...","[-0.62410355, 0.3616503, 0.9995014, -0.9757393...",0,0,...,0,0,0,1,0,0,0,0,1,0
10,4.4,84.0,2001,89,2002.0,Path.,"[0.5172503, 2.088545, 0.2931245, 0.16422777, 0...","[-0.6742195, 0.35496837, 0.9996903, -0.979621,...",0,0,...,0,0,0,0,0,0,0,0,0,0
11,7.2,132769.0,1997,113,1997.0,Mira.,"[0.42226052, 2.928343, 0.30139416, 0.3397494, ...","[-0.55780673, 0.31734014, 0.998963, -0.9590191...",0,0,...,0,0,0,0,0,0,0,0,1,0


### 1.2.2 Replacing 'studio' with a 'studio_pop' variable
As there are 498 different studios listed in this dataset, we deem it inefficient to create a dummy variable for every one of them. The main way in which we find it logical for a studio to influence a movie's gross revenue is through its popularity, i.e. how many movies it has produced in this dataset. An alternative in the same vein could have been to create dummy variables only for the most popular studios.

In [6]:
popularities = df['studio'].value_counts()   # This function yields exactly what we want !

df['studio_pop'] = df['studio'].map(popularities)

df.drop(columns=['studio'], inplace=True)
df.head()

,ratings,n_votes,production_year,runtime,release_year,img_embeddings,text_embeddings,Sci-Fi,Adventure,Game-Show,...,Family,Musical,Action,Film-Noir,Documentary,Music,Fantasy,Drama,Sport,studio_pop
0,6.5,92937.0,2010,105,2010.0,"[0.25030804, 2.4058464, 1.0431569, 0.030648155...","[-0.6795498, 0.35658365, 0.9994932, -0.9793934...",0,1,0,...,0,0,0,0,0,0,0,1,0,6
2,5.9,1345.0,1978,127,1978.0,"[0.18073043, 0.24735461, 0.63652813, 0.2496522...","[-0.709996, 0.4233521, 0.99980927, -0.98892415...",0,0,0,...,1,0,0,0,0,0,0,1,1,83
3,6.6,4851.0,1994,105,1994.0,"[0.025015268, 0.9105338, 0.3878257, 0.3421247,...","[-0.7416838, 0.38435012, 0.9998453, -0.9874693...",0,0,0,...,0,0,0,0,0,0,0,1,1,57
4,4.1,549.0,1982,108,1982.0,"[0.19079691, 1.9068279, 0.29114372, 0.19527505...","[-0.65501904, 0.3845747, 0.9996712, -0.9766391...",0,0,0,...,0,0,0,0,0,0,0,1,1,83
5,7.8,228296.0,1999,108,2000.0,"[0.41613612, 1.580189, 0.99092495, 0.55354017,...","[-0.77542937, 0.4372149, 0.9999121, -0.9899120...",0,0,0,...,0,0,1,0,0,0,0,0,0,36


### 1.2.3 Manipulation of the 'production_year' and 'release_year' variables
We discard the production year, as this year sometimes largely predates the release year : indeed, the movie might have taken a long time to finally be released in the US. Furthermore, the release year may affect a movie's gross revenue through economical events influencing the attendance of cinemas. We consider this to be far less true for the production year, as the economy of that year only (potentially) affected the production, not the public which ultimately is the one generating the gross revenue.
Finally, as a year is not a quantitative variable, we "quantitize" it by creating a variable "years_relased", counting the number of years that have passed since the movie's release.

In [7]:
df.drop(columns=['production_year'], inplace=True)

df['years_released'] = df['release_year'].map(lambda x : 2022-x)
df.drop(columns=['release_year'], inplace=True)
df.head()

,ratings,n_votes,runtime,img_embeddings,text_embeddings,Sci-Fi,Adventure,Game-Show,Short,Horror,...,Musical,Action,Film-Noir,Documentary,Music,Fantasy,Drama,Sport,studio_pop,years_released
0,6.5,92937.0,105,"[0.25030804, 2.4058464, 1.0431569, 0.030648155...","[-0.6795498, 0.35658365, 0.9994932, -0.9793934...",0,1,0,0,0,...,0,0,0,0,0,0,1,0,6,12.0
2,5.9,1345.0,127,"[0.18073043, 0.24735461, 0.63652813, 0.2496522...","[-0.709996, 0.4233521, 0.99980927, -0.98892415...",0,0,0,0,0,...,0,0,0,0,0,0,1,1,83,44.0
3,6.6,4851.0,105,"[0.025015268, 0.9105338, 0.3878257, 0.3421247,...","[-0.7416838, 0.38435012, 0.9998453, -0.9874693...",0,0,0,0,0,...,0,0,0,0,0,0,1,1,57,28.0
4,4.1,549.0,108,"[0.19079691, 1.9068279, 0.29114372, 0.19527505...","[-0.65501904, 0.3845747, 0.9996712, -0.9766391...",0,0,0,0,0,...,0,0,0,0,0,0,1,1,83,40.0
5,7.8,228296.0,108,"[0.41613612, 1.580189, 0.99092495, 0.55354017,...","[-0.77542937, 0.4372149, 0.9999121, -0.9899120...",0,0,0,0,0,...,0,1,0,0,0,0,0,0,36,22.0


### 1.2.4 Converting the 'runtime' entries to floats
Indeed, now that the \N are removed, all of them are floats encoded as strings

In [8]:
df['runtime'] = df['runtime'].map(lambda x : float(x))

### 1.2.5 Handling the image embeddings

In [9]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

img_embeddings_list = ['img_' + str(i) for i in range(2048)]

#print(df['img_embeddings'].map(lambda x : float((x[1:-1].split(","))[0])).head())

for i in range(2048):
    new_col = img_embeddings_list[i]
    df[new_col] = df['img_embeddings'].map(lambda x : float((x[1:-1].split(","))[i]))

df.drop(columns=['img_embeddings'], inplace=True)
df.head()

,ratings,n_votes,runtime,text_embeddings,Sci-Fi,Adventure,Game-Show,Short,Horror,War,...,img_2038,img_2039,img_2040,img_2041,img_2042,img_2043,img_2044,img_2045,img_2046,img_2047
0,6.5,92937.0,105.0,"[-0.6795498, 0.35658365, 0.9994932, -0.9793934...",0,1,0,0,0,0,...,0.115729,0.458049,0.921232,0.639395,0.699187,0.060539,0.158502,0.459635,0.723325,0.850279
2,5.9,1345.0,127.0,"[-0.709996, 0.4233521, 0.99980927, -0.98892415...",0,0,0,0,0,0,...,0.256043,0.336772,0.101175,0.242433,0.482843,0.187792,0.466467,0.023835,0.036051,0.127028
3,6.6,4851.0,105.0,"[-0.7416838, 0.38435012, 0.9998453, -0.9874693...",0,0,0,0,0,0,...,0.676794,0.628284,0.257829,0.205191,0.816652,0.058561,0.025909,0.131811,0.048419,1.148938
4,4.1,549.0,108.0,"[-0.65501904, 0.3845747, 0.9996712, -0.9766391...",0,0,0,0,0,0,...,1.111816,0.472659,0.235317,0.160041,0.160780,0.227784,0.689268,0.249997,0.283735,0.221155
5,7.8,228296.0,108.0,"[-0.77542937, 0.4372149, 0.9999121, -0.9899120...",0,0,0,0,0,0,...,0.510779,0.033777,1.377032,0.346408,0.565867,0.304034,0.121445,0.483406,0.374106,0.051325


### 1.2.6 Handling the text embeddings

In [10]:
text_embeddings_list = ['text_' + str(i) for i in range(768)]

for i in range(768):
    new_col = text_embeddings_list[i]
    df[new_col] = df['text_embeddings'].map(lambda x : float((x[1:-1].split(","))[i]))

df.drop(columns=['text_embeddings'], inplace=True)
df.head()

,ratings,n_votes,runtime,Sci-Fi,Adventure,Game-Show,Short,Horror,War,Animation,...,text_758,text_759,text_760,text_761,text_762,text_763,text_764,text_765,text_766,text_767
0,6.5,92937.0,105.0,0,1,0,0,0,0,0,...,0.555539,-0.173167,0.399331,-0.378869,-0.409315,-0.344284,-0.073671,0.999885,-0.495468,0.959063
2,5.9,1345.0,127.0,0,0,0,0,0,0,0,...,0.697693,-0.035854,0.356325,-0.388621,-0.344016,-0.322001,-0.085762,0.999952,-0.458579,0.976136
3,6.6,4851.0,105.0,0,0,0,0,0,0,0,...,0.692328,-0.009719,0.076474,-0.214084,-0.177248,0.025969,-0.112050,0.999954,-0.054416,0.977869
4,4.1,549.0,108.0,0,0,0,0,0,0,0,...,0.593455,-0.336046,0.406313,-0.384353,-0.334238,-0.377167,-0.131084,0.999879,-0.166202,0.951990
5,7.8,228296.0,108.0,0,0,0,0,0,0,0,...,0.754033,0.046537,0.173053,-0.280934,-0.137236,-0.114420,-0.137970,0.999974,0.074933,0.982905


In [11]:
df_after_embeddings = df.copy()   # to not have to rerun everything

## 1.3 Train-test splitting

In [12]:
Y = Y1[np.array((X1.runtime != '\\N'))]   # remove entries of Y that were removed from dataset

X_train, X_test, y_train, y_test = train_test_split(df,Y)
print(f'Training Samples shape: {X_train.shape} ; Training Target shape: {y_train.shape}\nTest Samples shape: {X_test.shape}; Test Target shape {y_test.shape}')

Training Samples shape: (2457, 2846) ; Training Target shape: (2457, 1)
Test Samples shape: (819, 2846); Test Target shape (819, 1)


## 1.4 Balance of influence: scaling

In [13]:
scaler = MinMaxScaler()   # StandardScaler might be required for some algorithms (ex: SVM) which require normality of the inputs

X_train = scaler.fit_transform(X_train, y_train)   # fit on training data and transform it; don't fit on the test data !
X_test = scaler.transform(X_test)   # use the same transformation for the test set

# Convert back to pandas dataframes !
X_train = pd.DataFrame(X_train, columns=df_after_embeddings.columns)
X_test = pd.DataFrame(X_test, columns=df_after_embeddings.columns)

X_train.head()

,ratings,n_votes,runtime,Sci-Fi,Adventure,Game-Show,Short,Horror,War,Animation,...,text_758,text_759,text_760,text_761,text_762,text_763,text_764,text_765,text_766,text_767
0,0.533333,0.111086,0.163121,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.788142,0.372118,0.509281,0.513350,0.557162,0.360403,0.541182,0.963265,0.431311,0.856770
1,0.426667,0.000725,0.134752,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.891102,0.485300,0.290463,0.424949,0.622690,0.432435,0.454479,0.977497,0.448916,0.899943
2,0.800000,0.304573,0.189716,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.670952,0.263131,0.852748,0.249184,0.251783,0.219996,0.709114,0.892871,0.204739,0.758407
3,0.613333,0.000970,0.180851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.759021,0.385508,0.786574,0.457220,0.466346,0.281163,0.467151,0.930780,0.327420,0.782321
4,0.693333,0.000959,0.278369,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.784980,0.464602,0.594765,0.361956,0.602553,0.572384,0.266610,0.973715,0.458093,0.823935


## 1.5 Outliers detection
Doesn't seem to make a lot of sense when we have more features than data : most data are considered outliers using the usual metrics. Let's keep all of them for now.

In [14]:
print(np.array((np.abs(scipy.stats.zscore(X_train)) < 3).all(axis=1)))

print((np.abs(scipy.stats.zscore(X_train))))
print(max(np.abs(scipy.stats.zscore(X_train).iloc[0])))
#[print(x) for x in scipy.stats.zscore(X_train.iloc[0])] 
#[print(np.abs(scipy.stats.zscore(X_train.iloc[i]))) for i in range(1)] 

#print(np.count_nonzero(np.array((np.abs(scipy.stats.zscore(X_train)) < 3).all(axis=1))))

print(y_train[np.array((np.abs(scipy.stats.zscore(X_train)) < 3).all(axis=1))].shape)

[False False False ... False False False]
       ratings   n_votes   runtime    Sci-Fi  Adventure  Game-Show     Short  \
0     0.392427  1.871255  0.036709  0.196116   0.357185   0.028542  0.114873   
1     1.092959  0.304560  0.447226  0.196116   0.357185   0.028542  0.114873   
2     1.358902  5.685949  0.490397  5.099020   2.799666   0.028542  0.114873   
3     0.132971  0.299737  0.339168  0.196116   0.357185   0.028542  0.114873   
4     0.658370  0.299942  2.002691  0.196116   0.357185   0.028542  0.114873   
...        ...       ...       ...       ...        ...        ...       ...   
2452  0.042162  0.318672  2.473700  0.196116   0.357185   0.028542  8.705243   
2453  1.446468  0.316811  2.140995  0.196116   0.357185   0.028542  0.114873   
2454  0.304861  0.294272  0.097200  0.196116   0.357185   0.028542  0.114873   
2455  0.917826  0.065078  0.097200  0.196116   0.357185   0.028542  0.114873   
2456  0.220538  0.064501  0.550889  0.196116   0.357185   0.028542  0.114873  

In [15]:
X_train.to_csv("X_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_test.to_csv("y_test.csv", index=False)

In [16]:
X_train.head()

,ratings,n_votes,runtime,Sci-Fi,Adventure,Game-Show,Short,Horror,War,Animation,...,text_758,text_759,text_760,text_761,text_762,text_763,text_764,text_765,text_766,text_767
0,0.533333,0.111086,0.163121,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.788142,0.372118,0.509281,0.513350,0.557162,0.360403,0.541182,0.963265,0.431311,0.856770
1,0.426667,0.000725,0.134752,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.891102,0.485300,0.290463,0.424949,0.622690,0.432435,0.454479,0.977497,0.448916,0.899943
2,0.800000,0.304573,0.189716,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.670952,0.263131,0.852748,0.249184,0.251783,0.219996,0.709114,0.892871,0.204739,0.758407
3,0.613333,0.000970,0.180851,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.759021,0.385508,0.786574,0.457220,0.466346,0.281163,0.467151,0.930780,0.327420,0.782321
4,0.693333,0.000959,0.278369,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.784980,0.464602,0.594765,0.361956,0.602553,0.572384,0.266610,0.973715,0.458093,0.823935
